In [4]:
class StudentAnswer:
    ans = {}
    
    
student1 = StudentAnswer()    

In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
# </snippet_imports>

'''
Authenticate
Authenticates your credentials and creates a client.
'''
# <snippet_vars>
subscription_key = "d5d0e372874a4c11ba463a30abe853f1"
endpoint = "https://answer-sheet.cognitiveservices.azure.com/"
# </snippet_vars>
# </snippet_imports_and_vars>

# <snippet_client>
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [8]:
from PIL import Image, ImageDraw
read_image = open("../images/image0.png", "rb")
# image = Image.open(read_image)

# draw = ImageDraw.Draw(image)
# Call API with image and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(read_image, raw=True)
# Get the operation location (URL with ID as last appendage)
read_operation_location = read_response.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for the retrieval of the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(1)


Waiting for result...


In [10]:

from PIL import ImageOps
image = Image.open(read_image).convert('RGB')
draw = ImageDraw.Draw(image)




grayImage = ImageOps.grayscale(image)

colorHasher={}
counter=0
lineSum = 0
topbox = 0
bottombox = 0
# Print results, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            lineSum += line.bounding_box[0]
            if counter==0:topbox=line.bounding_box[1]
            bottombox=line.bounding_box[5]
            counter+=1

marginLineLeft = 10
breakLoop = False
for top in range(10,int(topbox)):
    if(breakLoop):
        break
    for left in range(10,int(lineSum)//counter+10):
        if sum(image.getpixel((left,top)))<500:
            marginLineLeft=left
            breakLoop = True
            break
breakLoop = False
for top in range(int(bottombox),image.height-10):
    if(breakLoop):
        break
    for left in range(10,int(lineSum)//counter+10):
        if sum(image.getpixel((left,top)))<500:
            marginLineLeft=left
            breakLoop = True
            break
# margin
student1.ans = {}
draw.line((marginLineLeft,0,marginLineLeft,image.height-1), fill=(255,0,0))
qno = 0 
for text_result in read_result.analyze_result.read_results:
    for line in text_result.lines:
        print(line.text)
        coordinates = ((line.bounding_box[0], line.bounding_box[1]),
                           (line.bounding_box[4], line.bounding_box[5]))
        draw.rectangle(coordinates, outline=(0,150,0))
        if(qno!=0):
            student1.ans[qno] += " "
            student1.ans[qno] += line.text


        if line.bounding_box[0]<marginLineLeft:
            coordinates = ((line.bounding_box[0], line.bounding_box[1]),
                           (line.bounding_box[4], line.bounding_box[5]))
            draw.rectangle(coordinates, outline=(255,0,0))

            qno = line.text.split()[0].replace('.','')
            if(qno not in student1.ans):
                student1.ans[qno] = line.text[len(qno)+1:]

image.show()
print(student1.ans)

Register No. 1518/02131
Course code : UISCS 927 R
Course name : Cloud Computing
Date : 11/02 / 2022
No. of pages : 15
PART - A
Cloud computing :
In cloud computing , the client does not have to
buy The resources and manage thing . Instead , the provider.
provides the resources as a Service which is much cheaper .
Essentiel characteristics.
# On demand self services
* Resource pooling
* Measured service
2. Infrastructure as a Service ( IaaS) : resources
of Virtual machines
* In memory ( RAM ) storage
Disk storage
of Network devices
{'2': ' Infrastructure as a Service ( IaaS) : resources of Virtual machines * In memory ( RAM ) storage Disk storage of Network devices'}
